In [1]:
import pandas as pd

In [2]:
#read in the data from csv
players = pd.read_csv('all_seasons.csv', index_col=0)
allstars = pd.read_csv('allstars.csv', index_col=0)

In [3]:
display(players[players['player_name'] == "Michael Jordan"])
print(players.season.min())

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
392,Michael Jordan,CHI,34.0,198.12,97.975872,North Carolina,USA,1984,1,3,...,29.6,5.9,4.3,13.4,0.042,0.132,0.331,0.567,0.208,1996-97
467,Michael Jordan,CHI,35.0,198.12,97.975872,North Carolina,USA,1984,1,3,...,28.7,5.8,3.5,8.5,0.047,0.125,0.336,0.533,0.174,1997-98
2208,Michael Jordan,WAS,39.0,198.12,97.975872,North Carolina,USA,1984,1,3,...,22.9,5.7,5.2,-0.3,0.027,0.163,0.361,0.468,0.295,2001-02
3055,Michael Jordan,WAS,40.0,198.12,97.975872,North Carolina,USA,1984,1,3,...,20.0,6.1,3.8,0.0,0.028,0.167,0.288,0.491,0.207,2002-03


1996-97


The players dataframe has stats from every season from every player starting from 1996 to now. The 2019-20 season is incomplete, due to the coronavirus. To predict the current season allstars mid season, we should make sure there are no full season counting stats in the data, since they cannot be compared between mid or full season. The only stat like this in the dataframe is gp - games played. It can be changed to a fraction of games played instead, to solve the problem.

The NBA is always changing, so using old data to train for current models might not be smart. Limitting the data will also make everything run faster. Initially I will only use the data from 2010 onwards, maybe I will experiment with using the older data aswell later.

Players with very low counting stats like pts/reb/ast or who appeared in very few games are unlikely to be allstars, we can check this later after labelling the data frame.

In [4]:
print(players.columns)

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'college', 'country', 'draft_year', 'draft_round',
       'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],
      dtype='object')


In [5]:
col_to_keep = ['player_name', 'team_abbreviation', 'age', 'player_height', 'player_weight', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season']
players = players[col_to_keep]

In [6]:
players = players[players.season >= '2010-11']
print(players.season.min())
print(len(players))

2010-11
4919


To actually label the data we need the allstars dataframe, which contains the all stats from every season, dating back to 1990.
It is also possible to creature features out of this data, for example to predict if a player is an allstar in 2019-20, it would be useful to know if they were an allstar in 2018-19. This is likely to be a very powerful feature, it will be interesting to see how a classifier performs both with and without using this feature. A similar feature would be the total amount of allstar selections the player had prior to the current season.

In [7]:
allstars.head()

,Player,Age,Tm,Lg,Season
Rk,,,,,
1,Bam Adebayo\adebaba01,22,MIA,NBA,2019-20
2,Giannis Antetokounmpo\antetgi01,25,MIL,NBA,2019-20
3,Devin Booker\bookede01,23,PHO,NBA,2019-20
4,Jimmy Butler\butleji01,30,MIA,NBA,2019-20
5,Anthony Davis\davisan02,26,LAL,NBA,2019-20


In [8]:
#Try to make a function that can fix the name strings, this is necessary to combine data from both tables.
def fix_name(name):
    split = name.find('\\')
    return name[:split]

test_name = 'Govert\\123'
print(test_name)
test = fix_name(test_name)
print(test)

#Actually this can also be done inline with a lambda function
allstars['Player'] = allstars.Player.apply(lambda x: x[:x.find('\\')])
    

Govert\123
Govert


In [9]:
display(allstars.head())

,Player,Age,Tm,Lg,Season
Rk,,,,,
1,Bam Adebayo,22,MIA,NBA,2019-20
2,Giannis Antetokounmpo,25,MIL,NBA,2019-20
3,Devin Booker,23,PHO,NBA,2019-20
4,Jimmy Butler,30,MIA,NBA,2019-20
5,Anthony Davis,26,LAL,NBA,2019-20


In [10]:
def is_allstar(name, season):
    df = allstars[(allstars.Player == name) & (allstars.Season == season)]
    return len(df)


players['allstar'] = players[['player_name','season']].apply(lambda x: is_allstar(x.iloc[0], x.iloc[1]), axis=1)

In [11]:
players.head()

,player_name,team_abbreviation,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,allstar
6226,Kirk Hinrich,ATL,30.0,193.04,86.18248,72,10.2,2.5,4.0,-9.5,0.011,0.090,0.171,0.543,0.221,2010-11,0
6227,Kwame Brown,CHA,29.0,210.82,122.46984,66,7.9,6.8,0.7,-7.3,0.102,0.228,0.151,0.550,0.041,2010-11,0
6228,Kobe Bryant,LAL,32.0,198.12,92.98636,82,25.3,5.1,4.7,7.7,0.035,0.135,0.350,0.548,0.258,2010-11,1
6229,Kosta Koufos,DEN,22.0,213.36,120.20188,50,3.2,2.6,0.1,-6.1,0.134,0.185,0.189,0.474,0.025,2010-11,0
6230,Kris Humphries,NJN,26.0,205.74,106.59412,74,10.0,10.4,1.1,-5.3,0.125,0.322,0.173,0.555,0.069,2010-11,0


In [12]:
print(players.allstar.value_counts())
print(len(allstars[allstars.Season >= '2010-11']))

0    4667
1     252
Name: allstar, dtype: int64
259


Oops, looks like we've missed some players. Upon inspecting the data frames, it seems the problem is that the players dataframe has ascii formatting and the allstars dataframe has unicode, so one of them contains accented characters and the other doesnt. Let's try to fix this...

In [13]:
import unidecode

def extract_word(text):
    oldstr = text
    newstr = unidecode.unidecode(text)
    if newstr != oldstr:
        print("Input Text::{}".format(oldstr))
        print("Output Text::{}".format(newstr))
    return newstr

allstars['Player'] = allstars.Player.apply(extract_word)

Input Text::Luka Dončić
Output Text::Luka Doncic
Input Text::Nikola Jokić
Output Text::Nikola Jokic
Input Text::Nikola Jokić
Output Text::Nikola Jokic
Input Text::Nikola Vučević
Output Text::Nikola Vucevic
Input Text::Goran Dragić
Output Text::Goran Dragic
Input Text::Kristaps Porziņģis
Output Text::Kristaps Porzingis
Input Text::Manu Ginóbili
Output Text::Manu Ginobili
Input Text::Manu Ginóbili
Output Text::Manu Ginobili
Input Text::Peja Stojaković
Output Text::Peja Stojakovic
Input Text::Peja Stojaković
Output Text::Peja Stojakovic
Input Text::Peja Stojaković
Output Text::Peja Stojakovic


In [14]:
players['allstar'] = players[['player_name','season']].apply(lambda x: is_allstar(x.iloc[0], x.iloc[1]), axis=1)
print(players.allstar.value_counts())
print(len(allstars[allstars.Season >= '2010-11']))


0    4660
1     259
Name: allstar, dtype: int64
259


Success! Now let's try if it is possible to filter out some players based on low counting stats or low games played.

In [23]:
statfilter = (players.pts + players.reb + players.ast) > 10
players = players[statfilter]
print(players.allstar.value_counts())

0    2616
1     259
Name: allstar, dtype: int64


In [22]:
test = players[players['gp'] > 20]
print(test.allstar.value_counts())

0    2495
1     257
Name: allstar, dtype: int64


Looks like filtering on games played is not very useful, it doesn't filter out that many rows of data and we also lose some actual all star seasons. Now let's add more features based on previous allstar seasons.